This is the common problem set for Homework 3 from the spring quarter Network Theory class at UC Davis taught by Prof. Raissa D'Souza. The original assignment is at [http://mae.engr.ucdavis.edu/dsouza/Classes/253-S16/hw3.pdf](http://mae.engr.ucdavis.edu/dsouza/Classes/253-S16/hw3.pdf). Source code for this notebook is on github at [https://github.com/camillescott/ucd-ecs253](https://github.com/camillescott/ucd-ecs253). 

In [1]:
%pylab inline
import numpy as np
import networkx as nx
import seaborn as sns
sns.set_style('ticks')
sns.set_context('poster')

Populating the interactive namespace from numpy and matplotlib


In [65]:
np.set_printoptions(precision=4, linewidth=100)

# Modularity Matrix

## Bisection of a binary undirected network

In [58]:
A = np.array( [[0, 1, 1, 1, 0, 0],
               [1, 0, 1, 0, 1, 0],
               [1, 1, 0, 0, 0, 1],
               [1, 0, 0, 0, 1, 1],
               [0, 1, 0, 1, 0, 1],
               [0, 0, 1, 1, 1, 0]] )
print(A)

[[0 1 1 1 0 0]
 [1 0 1 0 1 0]
 [1 1 0 0 0 1]
 [1 0 0 0 1 1]
 [0 1 0 1 0 1]
 [0 0 1 1 1 0]]


In [59]:
k = A.sum(axis=0)
B = A - np.outer(k, k.T) / A.sum()

In [60]:
print(B)

[[-0.5  0.5  0.5  0.5 -0.5 -0.5]
 [ 0.5 -0.5  0.5 -0.5  0.5 -0.5]
 [ 0.5  0.5 -0.5 -0.5 -0.5  0.5]
 [ 0.5 -0.5 -0.5 -0.5  0.5  0.5]
 [-0.5  0.5 -0.5  0.5 -0.5  0.5]
 [-0.5 -0.5  0.5  0.5  0.5 -0.5]]


In [ ]:
print(np.array_str())

In [69]:
def eigenvalue_communities(B):
    w, v = np.linalg.eig(B)
    print('Eigenvalues:', w)
    
    v_arrow = v[:,np.argmax(w)]
    print('Eigenvector for largest eigenvalue:', v_arrow)
    
    communities = np.where(v_arrow > 0, [1]*len(v_arrow), [2]*len(v_arrow))
    print('\nCommunities:')
    for i, community in enumerate(communities):
        print('Node {0}:'.format(i+1), 'Community', community)

In [70]:
eigenvalue_communities(B)

Eigenvalues: [ -2.0000e+00   1.9733e-16   1.0000e+00  -2.0000e+00  -2.6080e-16  -3.2816e-17]
Eigenvector for largest eigenvalue: [-0.4082 -0.4082 -0.4082  0.4082  0.4082  0.4082]

Communities:
Node 1: Community 2
Node 2: Community 2
Node 3: Community 2
Node 4: Community 1
Node 5: Community 1
Node 6: Community 1


## Bisection of a weighted undirected network

In [71]:
A = np.array( [[0, 1, 0, .5],
               [1, 0, .5, 0],
               [0, .5, 0, 1],
               [.5, 0, 1, 0]])
print(A)

[[ 0.   1.   0.   0.5]
 [ 1.   0.   0.5  0. ]
 [ 0.   0.5  0.   1. ]
 [ 0.5  0.   1.   0. ]]


In [72]:
m = A.sum() / 2
print(m)

3.0


In [73]:
k = A.sum(axis=0)
B = A - (np.outer(k, k.T) / (2*m))

In [74]:
eigenvalue_communities(B)

Eigenvalues: [ -1.5000e+00  -5.0000e-01   5.0000e-01  -9.0433e-17]
Eigenvector for largest eigenvalue: [ 0.5  0.5 -0.5 -0.5]

Communities:
Node 1: Community 1
Node 2: Community 1
Node 3: Community 2
Node 4: Community 2
